In [20]:
%store -r autopilot_job_name

In [21]:
%store -r s3_bucket_name
s3_bucket_name

'sagemaker-cookbook-bucket'

In [22]:
%store -r prefix
prefix

'chapter06/input'

In [23]:
import sagemaker
session = sagemaker.Session()

In [24]:
autopilot_job = sagemaker.AutoML.attach(autopilot_job_name, 
                                        sagemaker_session=session)
endpoint_name = 'endpoint-' + autopilot_job_name

In [25]:
%%time

autopilot_job.deploy(
    initial_instance_count=1, 
    instance_type='ml.m5.large', 
    endpoint_name=endpoint_name)

-----------------!CPU times: user 254 ms, sys: 12.4 ms, total: 266 ms
Wall time: 8min 32s


In [26]:
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name, 
                      sagemaker_session=session)

In [27]:
!mkdir -p tmp

In [28]:
!aws s3 cp s3://{s3_bucket_name}/{prefix}/test_data.csv tmp/test_data.csv 

download: s3://sagemaker-cookbook-bucket/chapter06/input/test_data.csv to tmp/test_data.csv


In [29]:
import pandas as pd

test_data = pd.read_csv("tmp/test_data.csv")

In [30]:
test_data

,label,a,b,c,d
0,0,-13.956043,3.851976,3,84
1,0,-4.690871,8.672612,89,58
2,1,9.064805,2.775964,-4,0
3,1,5.459070,3.609733,-19,53
4,0,-7.777093,4.053004,71,38
...,...,...,...,...,...
995,0,3.667547,14.248178,-15,40
996,1,11.253117,4.465519,59,67
997,0,0.121657,9.131986,-20,80
998,0,2.191145,7.505053,-9,-43


In [31]:
test_data_without_label = test_data.drop(columns=['label'])
test_data_without_label

,a,b,c,d
0,-13.956043,3.851976,3,84
1,-4.690871,8.672612,89,58
2,9.064805,2.775964,-4,0
3,5.459070,3.609733,-19,53
4,-7.777093,4.053004,71,38
...,...,...,...,...
995,3.667547,14.248178,-15,40
996,11.253117,4.465519,59,67
997,0.121657,9.131986,-20,80
998,2.191145,7.505053,-9,-43


In [32]:
test_rows = test_data_without_label[0:5].to_csv(index=False, header=False)
test_rows

'-13.956042970317096,3.8519760635213096,3,84\n-4.690870896335887,8.672612385871918,89,58\n9.064805071667976,2.7759644610962453,-4,0\n5.459070345355769,3.609733375044708,-19,53\n-7.777093143592117,4.0530040039591615,71,38\n'

In [33]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(test_rows)

{'predictions': [{'predicted_label': '0'},
  {'predicted_label': '0'},
  {'predicted_label': '1'},
  {'predicted_label': '1'},
  {'predicted_label': '0'}]}

In [34]:
predictor.delete_endpoint()